虽然K-pop产业(Korean popular music)在本世纪初才面世，它在过去几年里却经历了飞速的国际化发展。Google现任首席执行官Sundar Pichai在2020届毕业生演讲“You will Prevail”里提到“I certainly never thought I’d be sharing a virtual stage with a former President…a First Lady, a Lady Gaga, and a Queen Bey…not to mention BTS”，可见K-pop文化在世界娱乐行业内的影响，尤其是在如今的互联网信息时代。K-pop产业的发展也同时带动了相应粉丝经济的蓬勃发展，在这次实验里，我们将通过分析K-pop粉丝的信息和行为来了解此行业。<br>

本次实验有三种类型的数据：<br>

1. 仅文字描述（复杂）: 
每个输入值的大小写不一致，分割符号不一致，且有乱码和个人文字描述的存在。比如以下这一列的输入值：

|Which is your favourite K-Pop group?|
|----|
|BTS;EXO;BLACKPINK;MONSTA X;Iâ€™m a multi-stan|
|BTS;EXO;MONSTA X;svt. got7. ateez. nct.|
|BTS;MONSTA X;a.c.e / nct / shinee / berrygood|
|BTS;EXO;I like many other group both not well known and well known|

 
2. 文字描述+固定选项（中等）:
输入值既有格式不一的文字描述也有固定格式的选项。比如以下这一列的输入值：

|Has consuming K-Pop taken away most of your time?|
|----|
|Yes|
|No|
|If you mean spending the night watching BTS crack or try not to laugh, than yes|
|Not sure|


3. 仅固定选项（简单）:
输入值为固定格式的选项。比如下面这一列的输入值有且仅有四种情况：

|Is K-Pop popular in your country?|
|----|
|Yes|
|No|
|Its gaining popularity|
|I'm not sure|

在进行数据清洗时可以将这三类数据分开来清洗。

### 步骤1 安装并引入必要的库

In [ ]:
!pip install pandas==0.25.0
!pip install xlrd==1.2.0

In [ ]:
import pandas as pd
import re

### 步骤2 导入数据并进行初步的整理

查看数据格式，删除不需要的列，把其余每一列的列名简化。

In [ ]:
#读取Excel里的数据
df = pd.read_excel("KPOP_DATA.xlsx")     
#添加'Num'这个列
df['Num'] = [i for i in range(len(df))]    
col = list(df.columns.values)   
 #将添加的'Num'列从最后一列变为第一列
df = df[[col[-1]] + col[0:23]]          

In [ ]:
#把Timestamp这一列删除
df = df.drop(columns=['Timestamp'])   
#对其余每一列进行重命名
df = df.rename(columns={                   
        'Which is your favourite K-Pop group?':'fav_group',          
        'Is K-Pop popular in your country? ':'pop_by_country', 
        'Why do you like K-Pop?':"reason",
        'When did you start listening to K-Pop?':"first_year",
        'Do you listen to K-Pop male groups more than girl group songs?':"kpop_gender_pref",
        'How many hours do you spend listening to K-Pop?':"daily_music_hr",
        'Do you watch K-Pop Youtube videos?':"watch_mv_yn",
        'If you do watch K-Pop music videos, how long do you spend watching them?':"daily_mv_hr",
        'Would you say that you are obsessed with K-Pop?':"obsessed_yn",
        'What do you do to keep up with K-Pop news?':"kpop_news",
        'Has K-Pop led you to pursue any of the following?':"pursuit",
        'Has consuming K-Pop taken away most of your time?':"time_consume_yn",
        'Has consuming K-Pop affected your personal life in the following ways?':"life_changing",
        'Have you experienced any positive effects after consuming K-Pop? ':"pos_eff",
        'On an average, in one year, how much do you spend on K-Pop merchandise?':"merch_spent_yr",
        'If you do spend money on K-Pop merchandise, where do you get money from?':"money_obtain",
        'Do you attend K-Pop concerts?':"concert_yn",
        'What is the craziest thing you have done in your pursuit and love for K-Pop?':"crazy_thing",
        'How old are you?':"age", 
        'Which country are you from?':"domicile",
        'What is your profession?':"job",
         'What is your gender?':"gender"
         })                   

### 步骤3 清洗格式不一致的列

#### 3.1 用主要元素建立词典：<br>
查看数据可发现fav_group这一列的数据比较杂乱，group name的格式不一致（大小写，空格，笔误），也没有一致的分割符。在这一步先建立词典，列出组合名可能的表达形式。

In [ ]:
#建立组合名的词典，词典的键包含了问卷结果里出现次数较高的组合名，对应的值包含了该组合名在问卷结果里的各种写法
group_dict = {            
"BTS" : ["BTS"],
"MONSTA X" : ["MONSTAX", "MONSTA X"],
"EXO" : ["EXO"],
"BLACKPINK" : ["BLACKPINK", "BLACK PINK"],
"STRAY KIDS" : ["STRAY KIDS", "STRAYKIDS"],
"NCT" : ["NCT"],
"GOT7" : ["GOT7", "GOT 7"],
"SEVENTEEN" : ["SEVENTEEN", "SEVENTERN", "SEVEN TEEN"],
"ATEEZ" : ["ATEEZ"],
"SHINEE" : ["SHINEE"],
"TWICE":["TWICE"],
"MAMAMOO":["MAMAMOO"],
"BIGBANG":["BIGBANG","BIG BANG"]
}  


#### 3.2 建立新的DataFrame，填入0/1值：<br>
建立以调查对象为行，组合名为列的DataFrame。若调查对象喜欢组合则标1，反之标0。

In [ ]:
#建立一个新的空白dataframe
favgroup_df = pd.DataFrame()  
#建立第一列 
favgroup_df['Num'] = df['Num']  
#用组合名词典里的键建立其余的列
for tmp in group_dict.keys():  
    #lambda函数在被提到的组合处标1，未被提到标0
    favgroup_df[tmp] = df["fav_group"].apply(lambda x: 1 if any(keyword in x.upper() for keyword in group_dict[tmp]) else 0)


#### 3.3 定义函数：<br>
定义函数将未出现在词典里的组合列为other。

In [ ]:
#创建函数，对于没有出现在group_dict里的组合将其归类为other
def other_group(x): 
    #建立空列表  
    check_list = []          
    for tmp in re.split(';|; | & |, |,| / | and |,and ',x):
        #读取每个输入项，并在读取的同时把输入项在;|; | & |, |,| / | and |,and 处分割
        if all(all(keyword not in tmp.upper() for keyword in group_dict[group]) for group in group_dict.keys()):
            #每个输入项被分割后成为一个列表，检查列表里的每个元素大写后是否能匹配group_dict里的任何值
            check_list.append(True)
            #不能匹配任何值的话标True
        else:
            check_list.append(False)
            #否则标False
    
    if any(check_list):   
        #每个输入项被分割后，有任一元素无法与group_dict里的任何值匹配，输出1
        return 1         
    else:
        #否则输出0
        return 0         

#### 3.4 导出csv表格：

In [ ]:
#创建新的列，命名为Others，用上述定义的other_group函数对输入项是否为Others进行0/1标号。
favgroup_df["Others"] = df["fav_group"].apply(lambda x: other_group(x))  
#查看前五行
favgroup_df.head()
#导出csv表格用于制作可视化图表
favgroup_df.to_csv("favgroup.csv", index = False)

以上清洗的favgroup是格式较乱的一列，因为每个输入项都是简答题，所以格式多样复杂。接下来清洗的列相对整齐，它们的输入由固定格式的单项/多项选择题和简答题组成。

### 步骤4 清洗数据格式为“固定选项+文字描述”的列：

#### 4.1 清洗reason列的数据：

##### 4.1.1 列出调查结果里的主要reason

In [ ]:
#建立空白列表
reason_list = []
#读取reason这一列
tmp = df['reason'].to_list()
for reasons in tmp:
    for reason in reasons.split(';'):
        #原因与原因之间由分号隔开，便可以通过在分号处做分割来建立所有原因的列表
        reason_list.append(reason)   

#对每一种原因出现的次数进行计数（用这个方法可以把调查对象的文字描述也列出来看一下）
reason_count = pd.Series(reason_list).value_counts()   
#列出出现次数大于一次的原因
top_reasons_list = list(reason_count[reason_count>1].index)  

##### 4.1.2 处理固定选项：创建新的DataFrame，用原因列表里的元素作为列

In [ ]:
#建立一个新的空白dataframe
reason_df = pd.DataFrame() 
#建立第一列
reason_df['Num'] = df['Num']    
#调查对象的文字描述里带有“all”或“everything”的话要把此对象在每个原因处标1
all_list = ["all", "everything"]     
#用原因列表里的元素建立其余的列
for reason in top_reasons_list:     
    reason_df[reason] = df['reason'].apply(lambda x: 1 if (reason in x) or (any(keyword in x.lower() for keyword in all_list)) else 0)


##### 4.1.3 处理文字描述：建立词典，对文字描述进行归类

In [ ]:
#查看文字描述后，建立词典进行归类                     
reason_dict = { 
"Music uniqueness" : ["uniqueness", "voices", "lyrics", "catchy", "attractive", "music", "meaning", "meaningful", "uplifting", "inspirational"],
"Reduced anxiety and bring happiness" : ["help", "struggle", "tough", "depression", "suicidal", "happy", "happier", "fun"],
"High Quality Performance": ["performance", "phenomenal", "quality", "ethic"],
"Idols Personality" : ["personality","friendly" ,"creativity", "personalities", "respective", "fans", "talent"],
"Study Asian/Korean culture" : ["culture", "study", "learn"]
}
#用词典里的键创建reason_df表里的列
for reason in reason_dict.keys():
       reason_df[reason] = df['reason'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in reason_dict[reason]) else 0)
     

##### 4.1.4 处理未被归类的值：定义函数

In [ ]:
#定义函数，将之前未被归类的值归为others    
def other_reason(x):
    check_list = []
    for tmp in x.split(";"):
        if (all(all(keyword not in tmp.lower() for keyword in reason_dict[reason]) for reason in reason_dict.keys())) and \
      (all(reason not in tmp for reason in top_reasons_list)) and (all(value not in tmp.lower() for value in all_list)):
            check_list.append(True)
        else:
            check_list.append(False)
    if any(check_list):
        return 1
    else:
        return 0

reason_df["Others"] = df['reason'].apply(lambda x: other_reason(x))
#查看前五行
reason_df.head()
#导出csv
reason_df.to_csv("reason.csv", index = False)


#### 4.2 清洗kpop_news列的数据：

##### 4.2.1 列出调查结果里的主要news

In [ ]:
#建立空列表
news_list = []
#在分号处隔开，读取所有news
tmp = df["kpop_news"].to_list()
for news in tmp:
    for new in news.split(";"):
        news_list.append(new)

#统计每种news出现次数        
news_count = pd.Series(news_list).value_counts()
#列出出现次数大于10的news
top_kpopnews_list = list(news_count[news_count>10].index)

##### 4.2.2 处理固定选项：

In [ ]:
#三个固定选项主要是：youtube，social media groups和news sites。把文字描述中带有这几个关键词的描述归类进这三个固定选项里。
youtube_list = ["youtube"]
group_list = ["amino","reddit"]  
website_list = ["soompi", "allkpop"]   

#建立空的DataFrame
kpopnews_df = pd.DataFrame()
#建立第一列
kpopnews_df['Num'] = df['Num']
for kpopnews in top_kpopnews_list:
    if kpopnews == "Subscribe to Youtube channels":
        kpopnews_df[kpopnews] = df['kpop_news'].apply(lambda x: 1 if (kpopnews in x) or any(keyword in x.lower() for keyword in youtube_list) else 0)
    elif kpopnews == "Join K-pop social media groups":
        kpopnews_df[kpopnews] = df['kpop_news'].apply(lambda x: 1 if (kpopnews in x) or any(keyword in x.lower() for keyword in group_list) else 0)
    elif kpopnews == "Subscribe to K-Pop news sites":
        kpopnews_df[kpopnews] = df['kpop_news'].apply(lambda x: 1 if (kpopnews in x) or any(keyword in x.lower() for keyword in website_list) else 0)

##### 4.2.3 处理文字描述：

In [ ]:
#建立词典，对文字描述里更丰富的关键词进行归类
news_dict = {
"Twitter" : ["twitter", "tweet"],
"Chat": ["friend", "chat", "gc"],
"Instagram" : ["instagram", "feeds"],
"Soompi / Allkpop" : ["soompi"],
"Follow idols' or fanbase social media accounts" : ["follow", "fan"]
}

for news in news_dict.keys():
    kpopnews_df[news] = df["kpop_news"].apply(lambda x: 1 if any(keyword in x.lower() for keyword in news_dict[news]) else 0)

##### 4.2.4 处理未被归类的值：定义函数

In [ ]:
def other_news(x):
    check_list = []
    for tmp in x.split(';'):
        if all(all(keyword not in tmp.lower() for keyword in news_dict[news]) for news in news_dict.keys()) and \
           all(keyword not in tmp for keyword in top_kpopnews_list) and \
           all(keyword not in tmp.lower() for keyword in (youtube_list+website_list+group_list)):
            check_list.append(True)
        else:
            check_list.append(False)
    
    if any(check_list):
        return 1
    else:
        return 0

kpopnews_df["Others"] = df["kpop_news"].apply(lambda x: other_news(x))
kpopnews_df.head()
kpopnews_df.to_csv("kpop_news.csv", index = False)

#### 4.3 清洗pursuit列的数据：

##### 4.3.1 列出调查结果里的主要pursuit（固定选项+文字描述）

In [ ]:
#建立空白列表
pursuit_list = []
tmp = df["pursuit"].to_list()
for pursuits in tmp:
    #在分号处分割开，读取所有提到的pursuit
    for pursuit in pursuits.split(";"):
        pursuit_list.append(pursuit)   

#对不同pursuit出现次数的计数
pursuit_count = pd.Series(pursuit_list).value_counts()  
#把出现次数多于10次的pursuit提取出来，存放在一个list里
top_pursuit_list = list(pursuit_count[pursuit_count>10].index)   


##### 4.3.2 处理文字描述：

通过查看pursuit列数据可发现，用文字描述里的部分关键词可把不同的文字描述归类到几个主要的pursuit里。

In [ ]:
 #把文字描述中的关键词分类，归入相应列表中
all_list = ["all"]
travel_list = ["visit", "travel"]
art_list = ["fashion", "makeup", "make up", "art"]  
dance_list = ["dance", "dancing"]
music_list = ["music", "sing","instruments", "albums"]  
culture_list = ["language", "speak","culture","help","mind","learn korean"]    

#建立空白的dataframe
pursuit_df = pd.DataFrame()  
#建立第一个列
pursuit_df["Num"] = df["Num"]   
for pursuit in top_pursuit_list:    
    if pursuit == "Learning about Korean/Asian culture":
        pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+culture_list)) else 0)
    elif pursuit == "Learn about Korean fashion/makeup":
        pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+art_list)) else 0)
    elif pursuit == "Visit Korea":
        pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+travel_list)) else 0)
    elif pursuit == "Pursue dancing":
        pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+dance_list)) else 0)
    elif pursuit == "Learning music - either an instrument or singing":
        pursuit_df[pursuit] = df["pursuit"].apply(lambda x: 1 if (pursuit in x) or any(keyword in x.lower() for keyword in (all_list+music_list)) else 0)


##### 4.3.3 处理未被归类的值：定义函数

In [ ]:
#定义函数
def other_pursuit(x):
    check_list = []
    for tmp in x.split(";"):
        if all(keyword not in x for keyword in top_pursuit_list) and \
    all(keyword not in x.lower() for keyword in (all_list+travel_list+art_list+dance_list+music_list+culture_list)):
            check_list.append(True)
        else:
            check_list.append(False)
            
    if any(check_list):
        return 1
    else:
        return 0

pursuit_df["Others"] = df["pursuit"].apply(lambda x: other_pursuit(x))
pursuit_df.head()
pursuit_df.to_csv("pursuit.csv", index = False)


### 步骤5: 清洗数据格式整齐一致的列：

#### 5.1 清洗pop_by_country列

In [ ]:
#因为这一列的格式统一内容固定，只需要用groupby对其进行统计
df.groupby(["pop_by_country"])['Num'].count()


#### 5.2 清洗daily_mv_hr列

In [ ]:
 #建立词典
mymap = {'1 hour':1, '2-3 hours':2.5, '3-4 hours':3.5, 'More than four hours':4.5}   
#应用lambda函数
df = df.applymap(lambda s: mymap.get(s) if s in mymap else s)
#用中值填补空格
df["daily_mv_hr"].fillna(df["daily_mv_hr"].mode().values[0] , inplace = True)
#日mv观看时长人数统计
df.groupby(["daily_mv_hr"])['Num'].count()       

#### 5.3 清洗daily_music_hr列

In [ ]:
#把输入值为2020-07-24 00:00:00的格归类为Others
df["daily_music_hr"] = df["daily_music_hr"].astype(str).replace("2020-07-24 00:00:00", "Others")
all_list = ["all", "whole", "everyday", "hours on end", "most", "every hour"]
everytime_list = ["any", "chance", "anytime", "whenever"]
mood_list = ["depend","mood"]
ans1_list = ["2-4 hours"]
ans2_list = ["5 or more", "half of my day"]
ans3_list = ["less than an hour"]

#定义函数
def music_hour(x):
    if any(keyword in x.lower() for keyword in everytime_list):
        return "Whenever listen to music"
    elif any(keyword in x.lower() for keyword in mood_list):
        return "Depend on the mood"
    elif any(keyword in x.lower() for keyword in all_list):
        return "24/7"
    elif any(keyword in x.lower() for keyword in ans1_list):
        return "2-4 hours"
    elif any(keyword in x.lower() for keyword in ans2_list):
        return "5 or more"
    elif any(keyword in x.lower() for keyword in ans3_list):
        return "Less than an hours"
    else:
        return "Others"
#应用函数
df["daily_music_hr"] = df["daily_music_hr"].apply(lambda x: music_hour(x))
#不同时间段的统计
df.groupby(["daily_music_hr"])['Num'].count()